# Алгоритм Томпсона на батчах (без контекста)

1. На первом батче распределяем юзеров 50% на 50%.
2. Вероятность конверсии каждого варианта распределена по Beta распределению с $\alpha=1$,
$\beta=1$.
3. В конце каждого батча пересчитываем вероятности превосходства по точной формуле,
взятой отсюда https://www.johndcook.com//UTMDABTR-005-05.pdf
4. Распределяем трафик в пропорции вероятностей превосходства для каждого варианта
5. Останавливаем эксперимент при достижении определенной вероятности превосходства,
но не раньше определенного дня, чтобы учесть календарные факторы

Потенциальные проблемы:
- слишком рано отдаем трафик победителю
- из-за дисбаланса распределения трафика может быть больше успешных конверсий в этом варианте
(можно попробовать применить нормализацию)


***Реализуем алгоритм***

0. **Инициализация** - *BatchThompson(n_arms)*. Аргумент на вход: число вариантов сплита.
Здесь также инициализируются массивы для параметров Бета-распределений и вероятность превосходства = 0.5.

1. **Метод сплита** - *split_data()*. Исходя из вероятности превосходства вычисляем сплит по вариантам.
Возвращаем данные по конверсии на текущем батче для пересчета Бета-распределений.

2. **Метод изменения параметров распределения** - *.update_beta_params(data)*. Аргументы на вход: numpy массив со значениями конверсии по
каждому варианту. В случае неравномерного распределения по вариантам ставятся пропуски.
 - Проверяем, чтобы число столбцов совпадало с числом вариантов из инициализации.
 - Суммируем нули и единицы и обновляем параметры

3. **Метод пересчета** - *update_prob_super()*. Аргументов нет, так как учитывает измененные параметры
$\alpha$ (накопленное число успешных конверсий) и
$\beta$ (накопленное число неудачных конверсий) для всех вариантов.
 - Считаем по точной формуле
 - Выдаем массив из вероятностей превосходства

4. **Вероятность превосходства** - *prob_super_tuple()*. Аргументов нет, так как берем пересчитанные параметры.
5. **Критерий остановки** (*stopping_criterion*) - условия цикла while. Либо вероятность превосходства выше заданной
величины, либо закончились наблюдения.

In [41]:
import os
import numpy as np
from typing import List, Tuple
from scipy.stats import beta
from tqdm.notebook import tqdm
from AB_classic import get_size_zratio
from MAB import calc_prob_between

# Графики
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import warnings
warnings.filterwarnings("ignore")


def create_directory_plot(folder: str, file_name: str):
    directory_plots = 'Plot/Thompson/' + folder + "/"
    try:
        os.mkdir(directory_plots)
    except:
        pass
    beta_distr_plot = PdfPages(directory_plots + file_name + ".pdf")
    return beta_distr_plot


class BatchThompson:
    def __init__(self, experiment_name:str, p_list: List[float], batch_size_share: np.float):
        self.p_list = p_list
        self.n_arms = len(p_list)
        self.n_obs_every_arm = get_size_zratio(p_list[0], p_list[1], alpha=0.05, beta=0.2)
        self.batch_size_share = batch_size_share
        self.experiment_name = experiment_name

        self.alphas = np.repeat(1.0, self.n_arms)
        self.bethas = np.repeat(1.0, self.n_arms)
        self.probability_superiority_tuple = (0.5, 0.5)

       # Generating data
        np.random.seed(np.uint16(np.random.random(size=1) * 100).item())
        self.data = np.random.binomial(n=[1,1], p=self.p_list, size=(self.n_obs_every_arm, self.n_arms))

        print(f"Нужно наблюдений в каждую руку для выявления эффекта в классическом АБ-тесте: "
              f"{self.n_obs_every_arm}")


    def split_data_historic(self, cumulative_observations: List, batch_split_obs: List):
        """
        Split data in every batch iteration
        :param cumulative_observations: list with cumulative observations for every arm
        :param batch_split_obs: how many observation we must extract this iter
        :return:
        """
        n_rows, n_cols = np.max(batch_split_obs), self.n_arms
        data_split = np.empty((n_rows, n_cols))
        data_split[:] = np.nan
        for i in range(self.n_arms):
            data_split[:batch_split_obs[i], i] = \
                self.data[cumulative_observations[i] : cumulative_observations[i] + batch_split_obs[i], i]
        return data_split


    def split_data_random(self, batch_split_obs: np.array):
        """

        :param batch_split_obs: size for every arm
        :param probs: probability for conversion rate
        :return:
        """
        data_split = np.empty((np.max(batch_split_obs), self.n_arms))
        data_split[:] = np.nan
        for i in range(self.n_arms):
            data_split[:batch_split_obs[i], i] = np.random.binomial(n=1, p=self.p_list[i],
                                                                    size=batch_split_obs[i])
        return data_split


    def update_beta_params(self, batch_data: np.array, method:str):
        if method == "summation":
            self._alphas += np.nansum(batch_data, axis=0)
            self._bethas += np.sum(batch_data == 0, axis=0)
        elif method == "normalization":
            S_list =  np.nansum(batch_data, axis=0)  # number of successes in within batch
            F_list = np.sum(batch_data == 0, axis=0)
            M = batch_data.shape[0]
            K = self._n_arms

            adding_alphas = (M / K ) * (np.array(S_list) / (np.array(S_list) + np.array(F_list)))
            adding_bethas = (M / K ) * (1 - np.array(S_list) / (np.array(S_list) + np.array(F_list)))

            adding_alphas = np.nan_to_num(adding_alphas)
            adding_bethas = np.nan_to_num(adding_bethas)

            self._alphas += adding_alphas
            self._bethas += adding_bethas
        return self._alphas, self._bethas


    def update_prob_super(self, method_calc) -> Tuple:
        if method_calc == 'integrating':
            prob_superiority =  calc_prob_between(self.alphas, self.bethas)
            self.probability_superiority_tuple = (prob_superiority, 1 - prob_superiority)


    # def create_plots(self, beta_distr_plot):
    #     x = np.linspace(0, 1, 100)
    #     rv1 = beta(self.alphas[0], self.bethas[0])
    #     rv2 = beta(self.alphas[1], self.bethas[1])
    #     fix, ax = plt.subplots()
    #     ax.plot(x, rv1.pdf(x), label='control')
    #     ax.plot(x, rv2.pdf(x), label='testing')
    #     leg = ax.legend();
    #     plt.title(f"Вероятность превосходства в %: "
    #               f"{np.round(tuple(map(lambda x: x * 100, self.prob_superiority_tuple)), 1)}")
    #     beta_distr_plot.savefig()
    #     plt.close()


    def start_experiment(self):
        probability_superiority_step_list = []  # how share of traffic changes across experiment
        observations_step_list = []  # how many observations is cumulated in every step

        # Plots
        # folder, file_name = self.experiment_name, str(self.p1) + "_" + str(self.p2)
        cumulative_observations = np.repeat(0, self.n_arms)  # how many observations we extract every iter for every arm

        for i in tqdm(range(0, np.uint16(1 / (self.batch_size_share / self.n_arms)))):
            # Create batch for iteration
            batch_size = self.batch_size_share * self.n_arms * self.n_obs_every_arm
            batch_split_obs = np.round(np.array(batch_size) * self.probability_superiority_tuple).astype(np.uint16)  # get number of observations every arm
            cumulative_observations += batch_split_obs
            # batch_data = batchT.split_data_historic(cumulative_observations=cumulative_observations,
            #                                         batch_split_obs=batch_split_obs) # based on earlier generated distr
            batch_data = self.split_data_random(batch_split_obs)  # based on generate batch online

            # Updating all
            self.update_beta_params(batch_data)  # update beta distributions parameters
            self.update_prob_super(method_calc="integrating") # update probability superiority

            # Append for resulting
            probability_superiority_step_list.append(self.probability_superiority_tuple)
            observations_step_list.append(batch_split_obs)

            stopping_criterion = (np.max(self.probability_superiority_tuple) >= 0.99) | \
                                 (np.max(cumulative_observations) >  self.n_obs_every_arm)
            if stopping_criterion:
                break

        return np.round(probability_superiority_step_list, 3), observations_step_list

bts = BatchThompson("Experiment5", p_list=[0.4, 0.5], batch_size_share=0.05)
probability_superiority_steps, observations_step_list = bts.start_experiment()
print(f"Наблюдений в каждую руку во время эксперимента: "
      f"\n {np.cumsum(observations_step_list, axis=0)}")

print(f"Вероятности превосходства каждой руки во время эксперимента: "
      f"\n {probability_superiority_steps}")

Нужно наблюдений в каждую руку для выявления эффекта в классическом АБ-тесте: 384


  0%|          | 0/40 [00:00<?, ?it/s]

Наблюдений в каждую руку во время эксперимента: 
 [[ 19  19]
 [ 25  51]
 [ 31  84]
 [ 35 119]
 [ 38 155]
 [ 43 189]
 [ 49 221]
 [ 59 249]
 [ 68 279]
 [ 71 314]
 [ 74 350]
 [ 78 384]
 [ 80 421]]
Вероятности превосходства каждой руки во время эксперимента: 
 [[0.167 0.833]
 [0.147 0.853]
 [0.093 0.907]
 [0.066 0.934]
 [0.118 0.882]
 [0.163 0.837]
 [0.271 0.729]
 [0.23  0.77 ]
 [0.076 0.924]
 [0.066 0.934]
 [0.115 0.885]
 [0.042 0.958]
 [0.037 0.963]]


array([[ 19,  19],
       [ 26,  51],
       [ 37,  78],
       [ 50, 103],
       [ 68, 123],
       [ 77, 152],
       [ 86, 181],
       [ 89, 217],
       [ 90, 255],
       [ 91, 292],
       [ 96, 326],
       [ 99, 361],
       [100, 398]], dtype=uint64)